##### Copyright 2024 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Context Caching Quickstart

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Caching.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

This notebook introduces context caching with the Gemini API and provides examples of interacting with the Apollo 11 transcript using the Python SDK. For a more comprehensive look, check out [the caching guide](https://ai.google.dev/gemini-api/docs/caching?lang=python).

In [ ]:
!pip install -qU 'google-generativeai>=0.7.0'

In [ ]:
import google.generativeai as genai
from google.generativeai import caching

## Configure your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) for an example.

In [ ]:
from google.colab import userdata

genai.configure(api_key=userdata.get("GOOGLE_API_KEY"))

## Upload a file

A common pattern with the Gemini API is to ask a number of questions of the same document. Context caching is designed to assist with this case, and can be more efficient by avoiding the need to pass the same tokens through the model for each new request.

Start by uploading a large file with the [File API](https://github.com/google-gemini/cookbook/blob/main/quickstarts/File_API.ipynb).

In [ ]:
!wget -q https://storage.googleapis.com/generativeai-downloads/data/a11.txt
!head a11.txt

INTRODUCTION

This is the transcription of the Technical Air-to-Ground Voice Transmission (GOSS NET 1) from the Apollo 11 mission.

Communicators in the text may be identified according to the following list.

Spacecraft:
CDR	Commander	Neil A. Armstrong
CMP	Command module pilot   	Michael Collins
LMP	Lunar module pilot	Edwin E. ALdrin, Jr.


In [ ]:
document = genai.upload_file(path="a11.txt")

## Cache the prompt

Next create a [`CachedContent`](https://ai.google.dev/api/python/google/generativeai/protos/CachedContent) object specifying the prompt you want to use, including the file and other fields you wish to cache. In this example the `system_instruction` has been set, and the document was provided in the prompt.

In [ ]:
# Note that caching requires a frozen model, e.g. one with a `-001` version suffix.
model_name = "gemini-1.5-flash-001"

apollo_cache = caching.CachedContent.create(
    model=model_name,
    system_instruction="You are a NASA transcript analyst. You understand spaceflight and can provide important context to transcripts.",
    contents=[document],
)

apollo_cache

CachedContent(
    name='cachedContents/7k9fow3y6xx5',
    model='models/gemini-1.5-flash-001',
    display_name='',
    usage_metadata={
        'total_token_count': 323394,
    },
    create_time=2024-06-18 06:03:36.541562+00:00,
    update_time=2024-06-18 06:03:36.541562+00:00,
    expire_time=2024-06-18 07:03:35.195945+00:00
)

## Manage the cache expiry

Once you have a `CachedContent` object, you can update the expiry time to keep it alive while you need it.

In [ ]:
import datetime

apollo_cache.update(ttl=datetime.timedelta(hours=2))
apollo_cache

CachedContent(
    name='cachedContents/7k9fow3y6xx5',
    model='models/gemini-1.5-flash-001',
    display_name='',
    usage_metadata={
        'total_token_count': 323394,
    },
    create_time=2024-06-18 06:03:36.541562+00:00,
    update_time=2024-06-18 06:03:38.612328+00:00,
    expire_time=2024-06-18 08:03:38.593924+00:00
)

## Use the cache for generation

As the `CachedContent` object refers to a specific model and parameters, you must create a [`GenerativeModel`](https://ai.google.dev/api/python/google/generativeai/GenerativeModel) using [`from_cached_content`](https://ai.google.dev/api/python/google/generativeai/GenerativeModel#from_cached_content). Then, generate content as you would with a directly instantiated model object.

In [ ]:
apollo_model = genai.GenerativeModel.from_cached_content(cached_content=apollo_cache)

response = apollo_model.generate_content("Find a lighthearted moment from this mission")
print(response.text)

The transcript is full of technical jargon, but there's a moment of levity when Buzz Aldrin, the Lunar Module Pilot, responds to a request for a crew status report. 

**02 00 20 31 CMP**
If we're late in answering you, it's because we're munching sandwiches.

**02 00 20 36 CC**
Roger. I wish I could do the same here.

**02 00 20 40 CMP**
No. Don't leave the console!

**02 00 20 42 CC**
Don't worry. I won't.

**02 00 20 47 CMP**
FLIGHT doesn't like it.

**02 00 20 54 CMP**
How is FLIGHT today?

**02 00 20 58 CC**
Oh, he's doing quite well. 

This humorous exchange highlights the fact that even during a monumental mission like Apollo 11, crew members and ground control find time to lighten the mood and share a laugh. It's a reminder that even when facing incredible pressure, the human spirit still finds ways to connect and inject some humor. 



You can inspect token usage through `usage_metadata`. Note that the cached prompt tokens are included in `prompt_token_count`, but excluded from the `total_token_count`.

In [ ]:
response.usage_metadata

prompt_token_count: 323403
candidates_token_count: 283
total_token_count: 292
cached_content_token_count: 323394

You can ask new questions of the model, and the cache is reused.

In [ ]:
chat = apollo_model.start_chat()
response = chat.send_message("Give me a quote from the most important part.")
print(response.text)

The most important part of this transcript is when Neil Armstrong takes his "one small step" on the moon. Here is that quote: 

"THAT'S ONE SMALL STEP FOR (A) MAN, ONE GIANT LEAP FOR MANKIND."

This iconic quote is spoken at 04:13:24 in the transcript and marks the moment when humankind first set foot on the moon.



In [ ]:
response = chat.send_message("What was recounted after that?")
print(response.text)

After Neil Armstrong's iconic statement, "That's one small step for (a) man, one giant leap for mankind," he goes on to describe the lunar surface in more detail:

* **The surface is fine and powdery.** Armstrong describes the moon's surface as "fine and powdery" and says he can pick it up loosely with his toe. He also notes that it adheres in fine layers to his boots, similar to powdered charcoal.
* **Footprints are clearly visible.** He mentions seeing the footprints of his boots and the tread marks in the sand-like particles. 
* **The surface is easy to move around on.** Armstrong says there's no difficulty in moving around, and it's even easier than the simulations on Earth. 
* **The descent engine did not leave a crater.** He mentions that the engine has about 1 foot of clearance from the ground, and there's no significant crater from the landing.

He also briefly mentions seeing rays emanating from the descent engine, suggesting that the engine exhaust did affect the surface in a

In [ ]:
response.usage_metadata

prompt_token_count: 323497
candidates_token_count: 257
total_token_count: 360
cached_content_token_count: 323394

## Counting tokens

The `GenerativeModel` object can be used to count the tokens of a request in the same manner as a direct, uncached, model.

In [ ]:
apollo_model.count_tokens("How many people are involved in this transcript?")

total_tokens: 9

## Delete the cache

A cache object will be deleted automatically when it expires. You can also explicitly delete a cache object.

In [ ]:
print(apollo_cache.name)
apollo_cache.delete()

cachedContents/7k9fow3y6xx5
